In [5]:
import os
from dotenv import load_dotenv

from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import openai
import numpy as np

openai.api_key = os.getenv("OPENAI_API_KEY")


# Cargar variables de entorno
load_dotenv()

# Ubicación del archivo de la base de datos

loader = CSVLoader(file_path='../../data/train.csv')
questions = loader.load()
database_file = 'data/chroma.sqlite3'

if os.path.exists(database_file):

    db = Chroma(persist_directory='../data/', embedding_function=OpenAIEmbeddings(
        api_key=os.getenv("OPENAI_API_KEY")), collection_name='movies_chroma.db')
    db.get()
else:
    # Crear la base de datos desde documentos si no existe
    db = Chroma.from_documents(questions, OpenAIEmbeddings(api_key=os.getenv(
        "OPENAI_API_KEY")), persist_directory='../data/', collection_name='movies_chroma.db')
    # Guardar la base de datos
    db.persist()


In [18]:
# query it
query = "i have a problem with my java code, can you help me?"
docs = db.similarity_search(query)
print(len(docs))
#print elements in the list
for doc in docs:
    print(doc)

# print results only title 
print(docs[1])


4
page_content='Id: 39450987\nTitle: eclipse failing to print the output becouse of some error\nBody: Im starting leanining java but even why i did everything as the guy in tutorial it is saying that i ahve an error. \r\nthis is my code: \r\n\r\n  \r\n\r\n      class code{\r\n\tpublic static void main(String args[])\r\n\t{\r\n\t\tSystem.out.println("coding...");\r\n\t}\r\n      }\r\n\r\n    \r\n\r\n\r\nthe output is this : \r\n\r\n   \r\n\r\n    Exception in thread "main" java.lang.Error: Unresolved compilation problem: \r\n\r\n\tat code.code.main(code.java:2)\r\n\r\nHope u ll help me sort this out **thanks**.\nTags: <java><eclipse>\nCreationDate: 2016-09-12 13:03:58\nY: LQ_EDIT' metadata={'row': 10447, 'source': '../../data/train.csv'}
page_content='Id: 39450987\nTitle: eclipse failing to print the output becouse of some error\nBody: Im starting leanining java but even why i did everything as the guy in tutorial it is saying that i ahve an error. \r\nthis is my code: \r\n\r\n  \r\n\r\

In [23]:
# create chroma object and upsert documents formed in json 
import pandas as pd
import os
import dotenv
load_dotenv()
secret_string= os.getenv('OPENAI_TOKEN')

# Configurar el motor de OpenAI
embeddings = OpenAIEmbeddings(api_key=secret_string, model="text-embedding-3-large")

def get_embedding(text):
    query_result = embeddings.embed_query(text)
    return query_result

db2 = Chroma(persist_directory='../data/')

df = pd.read_csv('../../data/train.csv')
df = df[:1000]
docs = []

for index, row in df.iterrows():
    doc = {
        'id': index,
        'title': row['Title'],
        'body': row['Body'],
    }
    docs.append(doc)





In [ ]:
db2.upsert_documents(docs, embeddings)
db2.persist()
print("Done")

In [ ]:
import langchain
from langchain_openai import OpenAIEmbeddings
import json
import os

# Obtener las credenciales de la base de datos y de OpenAI
load_dotenv()
token = os.getenv('DATASTAX_TOKEN')
url_datastax = os.getenv('DATASTAX_URL')
secret_string= os.getenv('OPENAI_TOKEN')

# Configurar el motor de OpenAI
embeddings = OpenAIEmbeddings(api_key=secret_string, model="text-embedding-3-large")

def get_embedding(text):
    query_result = embeddings.embed_query(text)
    return query_result

#create embeddings for each column of my csv
import pandas as pd
df = pd.read_csv('../../data/train.csv')

# Create a new column with the embeddings for title and another for body use my function  get_embedding
df['title_embedding'] = df['Title'].apply(lambda x: get_embedding(x))
df['body_embedding'] = df['Body'].apply(lambda x: get_embedding(x))

# Save the new dataframe with the embeddings
df.to_csv('../../data/train_embeddings.csv', index=False)


